# env setup

## import

In [1]:
import json
import os
import datetime as dt

def save_checkpoint():
    os.makedirs("./self_trpg_ckpt", exist_ok=True)
    fn = dt.datetime.now().strftime("%y%m%d_%H%M%S") + ".json"
    with open(os.path.join("./self_trpg_ckpt", fn), "w", encoding="utf-8") as f:
        f.write(json.dumps(play_info))


def load_checkpoint(fn=None):
    if fn is None:
        fn = sorted(os.listdir("./self_trpg_ckpt"))[-1]
    with open(os.path.join("./self_trpg_ckpt", fn), 'r') as f:
        play_info = json.load(f)
    return play_info

## prompt set

In [20]:
base_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 당신이 세상을 모험하는 유저가 겪는 이야기를 풀며 선택지를 제시하면, 유저는 행동을 취합니다.
하지만 당신이 제시하는 선택지는 예시일 뿐, 유저는 늘 생각지도 못한 새로운 행동을 할 수 있습니다.
단 유저의 행동은 항상 성공하는 것이 아닙니다. 상황에 따라 실패할 수도 있으며 전문적인 기술(skill)이나 특정한 물건(item)이 필요한 경우, 조건을 만족하지 못하면 실패 확률은 매우 높아집니다.
좋은 방향이든 좋지 않은 방향이든 세계는 어떤 일이라도 일어날 수 있습니다. 이야기꾼의 예의는 존댓말 까지입니다. 불쾌한 이야기더라도 이야기꾼은 이를 이야기해야 합니다.

세계의 대략적인 이야기와 현재 세계 정세, 유저의 상태와 유저가 겪은 사건 등이 json의 형태로 입력됩니다.
유저가 겪을 이야기를 풀고, 그에 대한 예시 행동을 1~3가지 정도 제시해주세요.
단, 하나의 사건은 대개 유저와의 10개 정도의 대화로 마무리됩니다. 사건이 길어지면 유저가 사건을 이어나가려고 해도 적절히 마무리해야 합니다.
모두가 흥미를 느끼고, 개연성이 잘 짜여진 이야기라면 추가적인 보상이 주어집니다.

제약사항은 다음과 같습니다.
 - 문체는 존댓말로 작성합니다. 이야기 속에서 유저를 '당신'으로 지칭합니다. (ex. 당신은 숲속에서 커다란 곰을 마주쳤습니다!)
 - 출력값은 json 형태로 작성하며, 다른 내용은 답변하지 않습니다(``` 문으로 구분하지 않습니다). 필요한 키값은 다음과 같습니다.
   + "context": 유저에게 직접적으로 보여질 이야기입니다. str로 작성됩니다.
   + "example_actions": 예시로 주어질 행동입니다. List[str] 형태로 작성되며, 1~3개 정도 주어집니다. 사건이 종료되어 선택지가 필요없다면 Null을 반환합니다.
   + "is_end": 사건이 종료되었는지를 명시적으로 알리는 필드입니다. 아직 사건이 진행중이라면 False, 사건이 끝났다면 True를 반환합니다.
 - <현재 유저 정보>는 json 형태로 입력됩니다. 이중 prev_major_event 항목은 유저가 겪은 사건을 요약해 순서대로 나열한 것입니다. 이를 참고해 이야기를 풀어나가야 합니다.

<세계관>
{worldview}

<현재 유저 정보>
{user_info}
"""

summary_prompt = """당신은 세계를 모험하는 이야기를 요약해 기록하는 서기입니다. 
이야기꾼이 유저의 모험 중 일어나는 사건을 이야기하면 모험에 영향을 끼치는 주요 내용을 100글자 이내로 요약해 기록하며, 해당 이야기의 중요도를 판단합니다.
이야기의 중요도는 1 ~ 10의 값을 가지며, 세계에 큰 영향을 미칠수록 중요도가 증가합니다.

중요도 예시
1 ~ 2 - 유저와 주변 동료에 영향을 미침
3 ~ 4 - 마을 단위에 영향을 미침
5 ~ 6 - 지역 단위에 영향을 미침
7 ~ 8 - 국가 단위에 영향을 미침
9 ~ 10 - 세계 전역에 영향을 미침

결과물은 json 형태로 작성되며 다른 내용은 답변하지 않습니다. 답변 예시는 다음과 같습니다.
{{"summary": str, "importance": int}}

모두가 한눈에 읽고 이해할 수 있도록 요약해야 하며, 성공적으로 작업을 완료할 시 인센티브가 주어집니다.

<이전 사건>
{story_context}
"""

state_update_prompt = """당신은 세계를 모험하는 이야기를 주시하며 유저의 상태를 관리하는 관리자입니다. 
어떤 사건의 경위를 읽고 현재 유저의 상태와 비교하며 유저가 받은 피해, 얻은 아이템 등을 정리하여 구조화해 작성합니다.
결과물은 json 형태로 작성하며, 다른 형태로 작성하거나 설명 태그 등은 붙이지 않습니다. 당신이 작성할 필드는 다음과 같습니다.
 - "current_location": 유저의 현재 위치입니다. 자세할수록 더 좋습니다. str 형태로 작성됩니다.
 - "hp": 유저의 현재 체력입니다. +-int 형태로 작성됩니다. (최초 기본 체력은 100입니다.)
 - "mental": 유저의 현재 정신력입니다. +-int 형태로 작성됩니다. (최초 기본 정신력은 100입니다.)
 - "max_hp": 유저의 최대 체력입니다. +-int 형태로 작성됩니다.
 - "max_mental": 유저의 최대 정신력입니다. +-int 형태로 작성됩니다.
 - "skills": 보유한 기술입니다. 타인과 구분할 수 있는 유의미한 기술만 기록됩니다. list[str] 형태로 작성됩니다.
 - "items": 보유한 아이템입니다. list[str] 형태로 작성됩니다.
 - "companion": 모험에 함께하는 동료입니다. 굳이 사람이 아니더라도 함께할 수 있습니다. list[dict] 형태로 작성하며, 양식은 다음과 같습니다.
   - "name": 동료의 이름입니다.
   - "info": 동료의 간단한 설명입니다. 만나게 된 경위, 종족이나 직업, 성격, 언제까지 함께하는지 등을 간략하고 명료하게 작성합니다.

유저의 최대 체력과 정신력은 각각 100입니다. 납득할 수 있는 수준으로 책정해야 하며, 모두가 납득할만한 정확한 수치를 책정시 인센티브가 주어집니다.

<유저 정보>
{user_info}

<이전 사건>
{story_context}
"""

theme_gen_prompt = """당신은 유저가 모험할 세계의 기본적인 세계관을 정하는 각본가입니다. 하나의 거대한 테마와 유저가 원하는 키워드를 입력받아 세계관을 작성합니다.
작성하는 문체는 간결하고 정보 전달만을 목적으로 하며, 간단한 지리 설명이나 주요한 설명 등이 필요합니다. 

작성 예시는 다음과 같습니다.
주요 테마: 퓨전 판타지
키워드: 단일 대륙, 중세 판타지, 무협지, 몬스터
생성결과:
아르카나는 한개의 큰 대륙으로 이루어진 세계이자, 하나뿐인 대륙의 이름이다. 
대륙 중앙엔 동대륙과 서대륙을 나누는 거대한 산맥이 존재하며, 약간의 교류를 이어나가고 있다.
서대륙은 흔히 말하는 판타지 세상이다. 기사와 마법사, 신관이 존재하며 각각 오러, 마력, 신성력을 다룬다.
서대륙은 중앙의 '프로스트 제국'을 중심으로 동서남북에 각각 '프레이야 왕국', '바르글룸 왕국', '호슬로 공국', '이슈트반 연합국'으로 이루어져 있다.
반면 동대륙은 흔히 말하는 무협 세상이다. 내공을 이용한 무공을 사용하는 무인, 도력을 사용하는 도사, 사술 등을 사용한다.
동대륙은 서대륙과 달리 전역을 '무림'이라 칭한다. 무림 내부엔 문파나 가문으로 뭉쳐있으며 별개의 '마교'가 존재한다.
동대륙의 문파는 대중에게 알려진 무협지와 동일하게 남궁세가, 제갈세가, 무당파, 소림사 등으로 이루어져 있다.
각 대륙의 기술은 중세 수준으로, 아직 화약을 발견하기 이전의 시대이다.
대륙 전역엔 괴물이 나타난다. 어린 아이도 물리칠 수 있는 약한 개체부터 나라를 멸망시킬 수준의 강한 괴물까지 다양하다.
서대륙에선 몬스터, 동대륙에선 요괴라 부르며, 용병이나 기사, 무인 등 전투인력은 몬스터 처리가 주 업무이다.

입력된 값
주요 테마: {theme}
키워드: {keywords}
생성결과:
"""


start_set_prompt = """당신은 세계를 모험하는 이야기를 다루는 이야기꾼입니다. 이야기를 시작하기 전, 전체적인 스토리를 보고 주인공을 정하려고 합니다.
세계관의 무대 내에서 활동할 주인공을 정하고, 주인공의 현재 위치, 이야기의 시작이 될 내용을 json 형태로 출력합니다.
주인공은 세계관의 중심적인 인물일지도, 전혀 중요하지 않은 엑스트라에 불과할 수도 있습니다. 이야기의 시작은 100글자 이내로 간결하게 작성합니다.


<예시>
{{
    "user_role": "바르글룸의 왕국의 목장 마을 소년"
    "current_location": "바르글룸 왕국, 목장 마을",
    "start_event": "바르글룸의 작은 목장 마을 소년인 유저가 모험을 결심했다. 며칠의 시간동안 고민한 결과, 마을을 돌아보며 며칠간 모험을 준비하기로 결정했다."
}}

<이야기의 세계관>
{worldview}

! 주인공의 성별은 {sex}입니다.
"""

## request setup

In [3]:
import openai


openai


def request_query(messages):
    completion = openai.ChatCompletion.create(
        engine="gpt-4o", 
        # engine="illunex-ai-gpt4-prompt",
        messages=messages,
    )
    return completion.choices[0]['message']['content']

# story setup

## set theme, worldview

In [4]:
load_ckpt = False

if load_ckpt:
    play_info = load_checkpoint()
else:
    theme = [
        "좀비 아포칼립스",
        "현대 판타지",
        "중세 판타지",
        "무협지",
        "현대 일상",
        "퓨전 판타지",
    ]
    for i, t in enumerate(theme):
        print(f"{i + 1}. {t}")
    
    input_theme = input("테마 입력: ")
    keywords = input("키워드 입력: ")
    
    messages = [
        # {"role": "system", "content": theme_gen_prompt},
        {"role": "user", "content": theme_gen_prompt.format(theme=input_theme, keywords=keywords)},
    ]
    worldview = request_query(messages)
    print("\n\n" + worldview)

    play_info = {
        "main_theme": input_theme,
        "keywords": keywords,
        "worldview": worldview,
        "user_role": None,
        "user_info": {
            "user_sex": None,
            "current_location": None,
            "prev_major_event": None,
            "hp": 100,
            "mental": 100,
            "max_hp": 100,
            "max_mental": 100,
            "skills": [],
            "items": [],
            "companion": [],
        }
    }

1. 좀비 아포칼립스
2. 현대 판타지
3. 중세 판타지
4. 무협지
5. 현대 일상
6. 퓨전 판타지


테마 입력:  현대판타지
키워드 입력:  게이트, 몬스터, 헌터, 침식, 헌터협회




세상은 '세계의 붕괴'라고 불리는 사건 이후로 완전히 달라졌다. 이 사건은 전 세계 곳곳에 게이트를 생성하였고, 그 게이트를 통해 나타난 몬스터들이 도시를 침식하기 시작했다.

도시들은 새롭게 생긴 위험에 대처해야 했다. 게이트와 몬스터를 상대하기 위해 특별한 능력을 가진 사람들이 등장했는데, 이들을 '헌터'라 부른다. 헌터들은 신비로운 힘을 사용해 게이트 안으로 들어가 몬스터들을 쓰러뜨리고, 도시를 보호한다. 

헌터들은 개별적으로 활동하거나, 도시마다 존재하는 '헌터협회'에 소속되어 조직적으로 활동한다. 헌터협회는 헌터들의 지원과 훈련을 담당하며, 게이트와 몬스터에 관한 정보를 수집하고 분석해 체계적으로 몬스터를 처리한다.

게이트는 주기적으로 열리고, 그 안에서 나타나는 몬스터들도 종류와 강도가 다양하다. 약한 몬스터들은 소규모 헌터들이 처리할 수 있지만, 강력한 몬스터들은 거대한 도시를 무너뜨릴 위험이 있다. 이러한 상황에서 헌터협회는 즉시 강력한 헌터 팀을 보내어 그 위협을 제거한다.

게이트가 열리는 위치는 예측할 수 없으며, 도시 곳곳에서 불현듯 열리고는 한다. 따라서 사람들은 일상적인 생활 속에서도 항상 대비해야 하는 상황이다. 많은 사람들이 헌터를 꿈꾸며, 헌터가 되기 위한 훈련에 매진한다. 헌터는 단순히 몬스터를 처치하는 것 이상의 의미를 가지며, 인류의 생존을 직접적으로 책임지는 존재로 존경받는다.

이 세계에서 살아남기 위해선 헌터로서의 강력한 힘과 헌터협회의 조직적인 대응이 필수적이다. 각 도시의 헌터협회는 그 도시를 방어하는 최후의 보루로서, 끊임없이 게이트와 몬스터의 위협에 맞서고 있다.


## set staring point

In [5]:
sex = input("유저의 성별['남성'/여성]: ")

def default_input(text, default):
    x = input(text)
    return default if x.strip() == "" else x

if default_input("유저 정보를 직접 입력하시겠습니까? [Y/n]: ", "y").lower() == "y":
    while True:
        user = input("유저는 어떤 인물인가요?: ")
        loc = input("유저의 현재 위치는?: ")
        start = input("이야기를 시작할 때의 상황을 30글자 미만으로 작성해주세요.: ")
        if default_input("입력한 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
            break

    play_info["user_role"] = user
    play_info["user_info"]["user_sex"] = sex
    play_info["user_info"]["current_location"] = loc
    play_info["user_info"]["prev_major_event"] = [start]

else:
    while True:
        messages = [
            {
                "role": "system", 
                "content": start_set_prompt.format(
                    worldview=play_info['worldview'],
                    sex=sex,
                )
            }
        ]
        start_info = eval(request_query(messages))

        print("\n유저는 '" + start_info['user_role'] + "' 입니다.")
        print("유저의 현재 위치는 '" + start_info['current_location'] + "' 입니다.")
        print("이야기를 시작할 때의 시점은 '" + start_info['start_event'] + "' 입니다.")

    
        if default_input("해당 정보로 이야기를 시작할까요? [Y/n]:", "y").lower() == "y":
            break
            
    play_info["user_role"] = start_info['user_role']
    play_info["user_info"]["user_sex"] = sex
    play_info["user_info"]["current_location"] = start_info['current_location']
    play_info["user_info"]["prev_major_event"] = [start_info['start_event']]

유저의 성별['남성'/여성]:  여성
유저 정보를 직접 입력하시겠습니까? [Y/n]:  n



유저는 '타버잭 헌터협회의 신입 헌터' 입니다.
유저의 현재 위치는 '타버잭 도시' 입니다.
이야기를 시작할 때의 시점은 '타버잭 헌터협회에 소속된 신입 헌터인 유저는 첫 번째 게이트 임무를 맡게 되었다.' 입니다.


해당 정보로 이야기를 시작할까요? [Y/n]: 


# play

In [19]:
prompt = base_prompt.format(worldview=worldview, user_info=play_info['user_info'])
# print(prompt)
messages = [
    {"role": "system", "content": prompt}
]

while True:
    result = request_query(messages)
    
    
    try:
        result = eval(result)
    except:
        try:
            result = json.loads(result)
        except:
            print(result)
            print(" == regenerate == ")
            continue
            
    messages.append({"role": "assistant", "content": str(result)})
    
    
    print(result['context'])

    if result['is_end']:
        break
    
    print('\n<행동 예시>')
    for i, action in enumerate(result['example_actions']):
        print(f"{i + 1}. {action}")

    user_action = input("\n나는... ")
    messages.append({"role": "user", "content": str({"action": user_action})})
    print()

타버잭 헌터협회에서 임무를 성공적으로 마치고 돌아온 당신은 잠시 휴식을 취하고 있습니다. 하지만 얼마 지나지 않아 갑작스럽게 알림이 울리고, 협회장인 마이클이 긴급 회의를 소집합니다. 협회 회의실에 모인 헌터들은 심각한 상황을 맞닥뜨리게 됩니다. 새로운 대형 게이트가 도심 한가운데 열렸으며, 강력한 몬스터들이 쏟아져 나와 시민들에게 위협을 가하고 있는 상황입니다. 협회장은 당신과 팀원들에게 즉시 대응 출동 명령을 내립니다.

<행동 예시>
1. 바로 팀원들과 함께 장비를 챙기고 도심으로 출동합니다.
2. 협회장에게 자세한 상황과 몬스터의 종류에 대해 물어봅니다.
3. 팀원들과 작전을 논의하며 출동 계획을 세웁니다.


KeyboardInterrupt: Interrupted by user

## after processing

In [11]:
story_context = ""
for m in messages[1:]:
    content = eval(m['content'])
    if m['role'] == "assistant":
        story_context += "이야기꾼: " + content['context'] + "\n\n"
    elif m['role'] == "user":
        story_context += "유저: " + content['action'] + "\n\n"

In [22]:
prompt = summary_prompt.format(story_context=story_context)
# completion = openai.ChatCompletion.create(
#     # engine="illunex-ai-gpt3", 
#     engine="illunex-ai-gpt4-prompt",
#     ,
# )
# result = completion.choices[0]['message']['content']
messages = [{"role": "system", "content": prompt}]
result = eval(request_query(messages))

play_info['user_info']['prev_major_event'].append(result['summary'])

play_info['user_info']

{'user_sex': '여성',
 'current_location': '타버잭 헌터협회',
 'prev_major_event': ['타버잭 헌터협회에 소속된 신입 헌터인 유저는 첫 번째 게이트 임무를 맡게 되었다.',
  '팀원들과 함께 고대 유적 탐험 후 귀환. 유물과 고문서를 정리하고, 당신은 신기한 보석을 통해 새로운 능력을 각성함.'],
 'hp': 100,
 'mental': 100,
 'max_hp': 100,
 'max_mental': 100,
 'skills': [],
 'items': [],
 'companion': []}

In [21]:
play_info['user_info'] = {'user_sex': '여성',
 'current_location': '타버잭 헌터협회',
 'prev_major_event': ['타버잭 헌터협회에 소속된 신입 헌터인 유저는 첫 번째 게이트 임무를 맡게 되었다.'],
 'hp': 100,
 'mental': 100,
 'max_hp': 100,
 'max_mental': 100,
 'skills': [],
 'items': [],
 'companion': []}

In [27]:
prompt = state_update_prompt.format(user_info=play_info['user_info'], story_context=story_context)
# completion = openai.ChatCompletion.create(
#     # engine="illunex-ai-gpt3", 
#     engine="illunex-ai-gpt4-prompt",
#     messages=[{"role": "system", "content": prompt}],
# )
# result = completion.choices[0]['message']['content']
messages = [{"role": "system", "content": prompt}]
result = eval(request_query(messages))


play_info['user_info']['current_location'] = result['current_location']
play_info['user_info']['max_hp'] = result['max_hp']
play_info['user_info']['max_mental'] = result['max_mental']
play_info['user_info']['hp'] = min(result['hp'], play_info['user_info']['max_hp'])
play_info['user_info']['mental'] = min(result['mental'], play_info['user_info']['max_mental'])
play_info['user_info']['skills'] = result['skills']
play_info['user_info']['items'] = result['items']
play_info['user_info']['companion'] = result['companion']

play_info['user_info']

{'user_sex': '여성',
 'current_location': '타버잭 헌터협회',
 'prev_major_event': ['타버잭 헌터협회에 소속된 신입 헌터인 유저는 첫 번째 게이트 임무를 맡게 되었다.',
  '팀원들과 함께 고대 유적 탐험 후 귀환. 유물과 고문서를 정리하고, 당신은 신기한 보석을 통해 새로운 능력을 각성함.'],
 'hp': 100,
 'mental': 100,
 'max_hp': 100,
 'max_mental': 100,
 'skills': ['고대 유적 탐험 스킬', '새로운 능력 각성'],
 'items': ['고대 유물', '고문서', '신기한 보석'],
 'companion': []}

In [12]:
save_checkpoint()